## Tokenizer

In [1]:
import torch
import numpy as np

In [7]:
k = lambda x: x
k(8)
torch.randn(3,2,1)

tensor([[[1.3153],
         [0.6936]],

        [[0.9872],
         [0.8483]],

        [[0.9075],
         [0.8698]]])

In [8]:
import pickle
with open("valid.pkl", 'rb') as f:
    val = pickle.load(f,)

val[:5]

[{'de': 'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten',
  'en': 'A Republican strategy to counter the re-election of Obama'},
 {'de': 'Die Führungskräfte der Republikaner rechtfertigen ihre Politik mit der Notwendigkeit, den Wahlbetrug zu bekämpfen.',
  'en': 'Republican leaders justified their policy by the need to combat electoral fraud.'},
 {'de': 'Allerdings hält das Brennan Center letzteres für einen Mythos, indem es bekräftigt, dass der Wahlbetrug in den USA seltener ist als die Anzahl der vom Blitzschlag getöteten Menschen.',
  'en': 'However, the Brennan Centre considers this a myth, stating that electoral fraud is rarer in the United States than the number of people killed by lightning.'},
 {'de': 'Die Rechtsanwälte der Republikaner haben in 10 Jahren in den USA übrigens nur 300 Fälle von Wahlbetrug verzeichnet.',
  'en': 'Indeed, Republican lawyers identified only 300 cases of electoral fraud in the United States in a decade.'},
 {'de': 'Eins i

In [10]:
total = [dic[k] for dic in val for k in dic ]

In [6]:
from datasets import load_dataset

k = load_dataset("wmt14", "de-en")
k

import pickle
with open("valid.pkl", 'wb') as f:
    pickle.dump(k["validation"]["translation"], f)

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

...

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Dataset wmt14 downloaded and prepared to /home/user6/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer(BPE())
trainer = BpeTrainer(special_tokens = ["<unk>", "<sos>", "<eos>", "<pad>"])
tokenizer.train("val", trainer=trainer)

Exception: No such file or directory (os error 2)

In [5]:
tokenizer.get_vo

{}

## Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [15]:
b = 16  # Batch
l = 30  # Len of Sent
e = 100 # Embed dim

q = torch.randn(b, l, e)
k = torch.rand_like(q); v = torch.rand_like(q)
print(q.shape)

torch.Size([16, 30, 100])


In [43]:
score = torch.einsum("ble,ble->bl", q, k)
print(score.shape)
##mask
score = torch.softmax(score, axis=1)
o = torch.einsum("ble,bl->ble", v, score)
o.shape

torch.Size([16, 30])


torch.Size([16, 30, 100])

In [ ]:
k = 100
s = torch.tensor(range(k)).to(torch.float)
s_= 1/torch.sqrt(torch.tensor(k))
print(torch.softmax(s,0))
print(torch.softmax(s*s_,0))
s_=torch.sqrt(torch.tensor(4))


In [2]:
from src.Attention import SelfAttention
import torch 
batch = 20
sent_len = 40
sent_len2 = 60
embed = 200
n_head = 6
d_key = 100
d_hidden = 200
d_out = 200
a = torch.randn(batch, sent_len, embed)
b = torch.randn(batch, sent_len2, embed)
attend = SelfAttention(d_hidden, mask=True)
#attend(a,b,c).shape

In [2]:
W_q1 = torch.nn.Linear(embed, d_key*n_head, bias=False)
W_k1 = torch.nn.Linear(embed, d_key*n_head, bias=False)
W_v1 = torch.nn.Linear(embed, d_hidden*n_head, bias=False)
W_o1 = torch.nn.Linear(n_head*d_hidden, d_out, bias=False)

W_q2 = torch.randn(embed, n_head, d_key)
W_k2 = torch.randn(embed, n_head, d_key)
W_v2 = torch.randn(embed, n_head, d_hidden)
W_o2 = torch.randn(n_head, d_hidden, d_out)

In [3]:
print(a.shape)
q = torch.einsum("ble, ecd->blcd", a, W_q2)
k = torch.einsum("ble, ecd->blcd", a, W_k2)
v = torch.einsum("ble, ech->blch", a, W_v2)
shape = q.shape
#q = q.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
#k = k.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
#v = v.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
o = attend(q, k, v)
o = torch.einsum("blch, cho->blo", o, W_o2)
print(o.shape)

q = torch.randn(5,3,2,1)
k = torch.randn(5,3)
#q  k.unsqueeze(2)

torch.Size([20, 40, 200])
torch.Size([20, 40, 200])


In [14]:
from time import time
a = torch.randn(batch, sent_len, embed)
b = torch.randn(batch, sent_len2, embed)


### way1
s = time()
for i in range(100):
    q = W_q1(a); k = W_k1(a); v = W_v1(a)
    shape = q.shape
    print(shape)
    q = q.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
    k = k.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
    v = v.view(shape[0], shape[1], n_head, -1)#.transpose(0,2)
    print(v.shape)
    o = attend(q, k, v).view(shape[0], shape[1], -1)
    print(o.shape)
    #o = torch.einsum("blch, cho->blo", o, W_o2)
    o = W_o1(o)
    break

print(time()-s)

### way2
s = time()
for i in range(100):
    break
    q = torch.einsum("ble, ecd->blcd", a, W_q2)
    k = torch.einsum("ble, ecd->blcd", a, W_k2)
    v = torch.einsum("ble, ech->blch", a, W_v2)

    o = attend(q, k, v)
    o = torch.einsum("blch, cho->blo", o, W_o2)

print(time()-s)

torch.Size([20, 40, 600])
torch.Size([20, 40, 6, 200])
torch.Size([20, 40, 1200])
0.03194117546081543
0.0


In [18]:
ll = torch.nn.LayerNorm(200)
ll(o).shape

torch.Size([20, 40, 200])

In [3]:
from src.Attention import MultiHeadAttention

M = MultiHeadAttention(num_head=n_head, in_dim=embed, hidden_dim=d_hidden, key_dim=d_key, out_dim=d_out)

M(a).shape

torch.Size([20, 40, 200])

In [49]:
a = torch.tensor([
    [1,-1,-1],  #1
    [1,1,-1],   #2
    [1,1,1],    #3
    [-1,1,-1]   #4
])
b = torch.tensor([
    [1,1,1],    #3
    [1,-1,-1],  #1
    [-1,1,-1],  #4
    [1,1,-1]    #2
])

torch.einsum("ij,Ij->iI", a,b)

tensor([[-1,  3, -1,  1],
        [ 1,  1,  1,  3],
        [ 3, -1, -1,  1],
        [-1, -1,  3,  1]])

In [7]:
torch.ones(3,3).tril(diagonal=-1)

tensor([[0., 0., 0.],
        [1., 0., 0.],
        [1., 1., 0.]])

In [33]:

samp_q = torch.randn(4,3,2,1)
samp_k = torch.randn(4,4,2,1)
samp_v = torch.randn(4,4,2,2)
samp_s = torch.einsum("blcd,bmcd->bclm", samp_q, samp_k) / 3
mask = torch.ones(3,4).to(torch.int).tril()
samp_s = samp_s.masked_fill(mask == 0, -100)
samp_s = torch.softmax(samp_s, axis=3)
samp_o = torch.einsum("bmch,bclm->blch", samp_v, samp_s)
samp_s[0, 1, :, :]
samp_o[0]

tensor([[[-1.5683,  0.9822],
         [ 1.3979,  1.0357]],

        [[-0.7038,  1.0138],
         [ 0.0685, -0.0544]],

        [[-0.5437,  0.7363],
         [-0.1208, -0.5742]]])

In [2]:
import torch
from src.Layer import Encoder, Decoder
from src.Model import Transformer

batch = 20
vocab_size = 100
sent_len = 40
sent_len2 = 60
d_model = 200
n_head = 6
d_key = 100
d_value = 200
d_feed = 1024
d_out = 200

n_layer = 6
each = True

"""samp = torch.randn(batch, sent_len, embed)
samp2 = torch.randn(batch, sent_len2, embed)

sampEnc = Encoder(num_layers = n_layer, num_head=n_head, model_dim=embed, value_dim=d_value, key_dim=d_key, feed_dim=d_feed, return_each=each)
sampDec = Decoder(num_layers = n_layer, num_head=n_head, model_dim=embed, value_dim=d_value, key_dim=d_key, feed_dim=d_feed, get_each=each)

temp = sampEnc(samp)
print(temp.shape) if not each else 0
temp = sampDec(samp2, temp)
print(temp.shape)"""

model = Transformer(
    vocab_size=vocab_size, 
    max_len=sent_len + sent_len2,
    num_layers=n_layer, 
    num_head=n_head,
    model_dim=d_model,
    value_dim=d_value, 
    key_dim=d_key,
    feed_dim=d_feed,
    dropout=.1, send_each=False, embed_share=True)

samp = torch.randint(0, vocab_size, (batch, sent_len))
samp2 = torch.randint(0, vocab_size, (batch, sent_len2))

print(model(samp,samp2).size())

model.predict(samp, max_len=20).size()

torch.Size([20, 60, 100])


torch.Size([20, 20, 100])

In [74]:
torch.sin(torch.tensor(3))

size = 10
dim = 3

k = torch.arange(size).repeat(dim,1).transpose(1,0)
k #/ torch.tensor([1,-1])
scale = torch.arange(dim)/dim
scale = 1/ torch.pow(10000, scale)
s = torch.sin(k * scale)
c = torch.cos(k * scale)

o = torch.cat((s,c), axis=0)
o = o.reshape(size, -1)
o.size()
## shape = size * 2*dim
Emb = torch.nn.Embedding(num_embeddings=5, embedding_dim=2*dim)

samp = torch.tensor([[0,1,2],[3,4,0]])
print(samp.size())
Emb(samp) + o[:3, :]

torch.Size([2, 3])


tensor([[[ 0.5965,  0.7076, -0.9089,  0.3866,  0.7745,  1.6758],
         [ 2.5311,  0.4324, -0.2381, -0.4608, -0.6070,  0.0152],
         [-1.2052, -1.2878, -0.9749,  0.6621, -0.7977,  1.7467]],

        [[-0.6743,  0.2969,  0.0333,  2.3706, -0.0154, -1.5639],
         [ 0.1228,  1.1158, -0.0349,  0.6484,  0.7532,  0.1696],
         [-0.1603,  0.8922, -0.9002, -1.4138,  0.9581,  1.6844]]],
       grad_fn=<AddBackward0>)

In [18]:
from src.SubLayer import PositionalEmbedding as Embedding
import torch

Emb = Embedding(vocab_size=100, embedding_dim=256, max_len=200)
samp = torch.randint(0, 5, (3,8))
Emb(samp).shape

torch.Size([3, 8, 256])

In [7]:
#Emb.parameters
#torch.sum(Emb.trigonal > 0)
#lin = torch.nn.Linear(6, 5)

torch.mean(torch.cat([torch.rand(3,2).unsqueeze(0) for i in range(3)], dim=0), dim=0)

k = torch.randint(high=3, size=(3,2))
print(k)
k.argmax(dim=-1)

torch.save(k, f='sample.pt')

tensor([[0, 0],
        [0, 1],
        [1, 0]])


In [8]:
f = torch.load("sample.pt",)
f

tensor([[0, 0],
        [0, 1],
        [1, 0]])